### Save / Loading Models 

Tensorflow provides a few ways of saving/loading models. The easiest way is to use a checkpoint. Though, this really useful while you are training your network. When you are ready to deploy or hand out your network to others, you don't want to pass checkpoints around as they contain a lot of unnecessary information, and it also requires you to still write code to create your network. Instead, you can create a protobuf which contains the definition of your graph and the model's weights. Let's see how to do both:

#### read:
https://github.com/pkmital/CADL/blob/master/session-3/lecture-3.ipynb

#### Checkpoint

In [91]:
import tensorflow as tf
import os
import math

In [92]:
#### just create a random variable 
def build_graph():
    ## create a sample graph
    with tf.name_scope("operation"):
        w1 = tf.placeholder("float", name="w1")
        w2 = tf.placeholder("float", name="w2")
        b1= tf.Variable(2.0,name="bias")
        #Define a test operation that we will restore
        w3 = tf.add(w1,w2)
        logits = tf.multiply(w3,b1)
        logits = tf.identity(logits,name="logits")
    return w1,w2,logits

In [93]:
tf.reset_default_graph()

In [94]:
## run build session 
with tf.Session() as sess:
    w1,w2,logits = build_graph()
    ## initialize option
    init_all_op = tf.global_variables_initializer()
    # Initializes all the variables.
    sess.run(init_all_op)
    # Runs to logit.
    feed_dict ={w1:4,w2:8}
    sess.run(logits,feed_dict)
    # Creates a saver.
    saver = tf.train.Saver()
    #### save model in ckpt
    save_path = saver.save(sess, path)
    print("Model saved in file: %s" % save_path)

Model saved in file: ./ckpt/model.pb


#### load ckpt file 

In [95]:
tf.reset_default_graph()

In [96]:
graph = tf.Graph()
with tf.Session(graph = graph) as sess:
    new_saver = tf.train.import_meta_graph(save_path+'.meta')
    new_saver.restore(sess, save_path)
    #saver.restore(sess,tf.train.latest_checkpoint('./'))
    
    ## variables names list 
    node_names = [n.name for n in graph.as_graph_def().node]
    #print(node_names[-40:])
    ## print out a particular variable 
    w1 = graph.get_tensor_by_name('operation/w1:0')
    w2 = graph.get_tensor_by_name('operation/w2:0')
    logits = graph.get_tensor_by_name('operation/logits:0')
    
    ## get and print logits 
    feed_dict = {w1:1,w2:2}
    print(sess.run(logits,feed_dict))
    

INFO:tensorflow:Restoring parameters from ./ckpt/model.pb
6.0


### Protofuf

In [97]:
tf.reset_default_graph()

In [103]:
path='./ckpt'
fname = 'model.pb'
output_node = ['operation/w1','operation/w2','operation/logits']
## run build session 
graph=tf.Graph()
with tf.Session(graph=graph) as sess:
    w1,w2,logits = build_graph()
    init_all_op = tf.global_variables_initializer()
    sess.run(init_all_op)
    feed_dict ={w1:4,w2:8}
    sess.run(logits,feed_dict)

    graph_def = tf.graph_util.convert_variables_to_constants(
        sess, sess.graph_def, output_node)

    tf.train.write_graph(graph_def,path,fname,as_text=False)
    print("Model saved in file: %s" % path+'/'+fname)

INFO:tensorflow:Froze 1 variables.
Converted 1 variables to const ops.
Model saved in file: ./ckpt/model.pb


##### load pb

In [104]:
path='./ckpt/model.pb'
tf.reset_default_graph()
with open(path, mode='rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

In [105]:
with tf.Session() as sess:
    w1 = sess.graph.get_tensor_by_name('operation/w1:0')
    w2 = sess.graph.get_tensor_by_name('operation/w2:0')
    logits = sess.graph.get_tensor_by_name('operation/logits:0')
    
    ## get and print logits 
    feed_dict = {w1:1,w2:2}
    print(sess.run(logits,feed_dict))

6.0
